In [17]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/Sumi2089/Datasheet/refs/heads/main/thesis5.csv')

In [18]:
X0 = []
X1 = []
X2 = []
X3 = []
X4 = []
X5 = []
X6 = []
y = []

for i in range(0, df.shape[0]-24):
    X0.append(df.iloc[i:i+24, 0])
    X1.append(df.iloc[i:i+24, 1])
    X2.append(df.iloc[i:i+24, 2])
    X3.append(df.iloc[i:i+24, 3])
    X4.append(df.iloc[i:i+24, 4])
    X5.append(df.iloc[i:i+24, 5])
    X6.append(df.iloc[i:i+24, 6])
    y.append(df.iloc[i+24, 4])

X0, X1, X2, X3, X4, X5, X6, y = np.array(X0), np.array(X1), np.array(X2), np.array(X3), np.array(X4), np.array(X5), np.array(X6), np.array(y)
y = np.reshape(y, (len(y),1))


In [19]:
#Normalize the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
X0 = scaler.fit_transform(X0)
X1 = scaler.fit_transform(X1)
X2 = scaler.fit_transform(X2)
X3 = scaler.fit_transform(X3)
X4 = scaler.fit_transform(X4)
X5 = scaler.fit_transform(X5)
X6 = scaler.fit_transform(X6)
y = scaler.fit_transform(y)
X = np.stack([X0, X1, X2, X3, X4, X5, X6], axis=2)

In [32]:
# Train set and test set
X_train, X_test = X[ :14616], X[14712:14736]
y_train, y_test = y[ :14616], y[14712:14736]

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras import optimizers
input_shape=([X_train.shape[1], X_train.shape[2]])

def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(100, return_sequences=False, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='relu'))

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model

In [ ]:
lstm_model = create_lstm_model([X_train.shape[1], X_train.shape[2]])

In [76]:
lstm_model.fit(X_train, y_train, validation_split=0.2, epochs=5, batch_size=16)

Epoch 1/5
731/731 ━━━━━━━━━━━━━━━━━━━━ 25s 35ms/step - loss: 0.0013 - mae: 0.0257 - val_loss: 0.0013 - val_mae: 0.0237
Epoch 2/5
731/731 ━━━━━━━━━━━━━━━━━━━━ 41s 35ms/step - loss: 9.9681e-04 - mae: 0.0229 - val_loss: 0.0014 - val_mae: 0.0256
Epoch 3/5
731/731 ━━━━━━━━━━━━━━━━━━━━ 29s 40ms/step - loss: 9.1763e-04 - mae: 0.0220 - val_loss: 0.0014 - val_mae: 0.0264
Epoch 4/5
731/731 ━━━━━━━━━━━━━━━━━━━━ 25s 35ms/step - loss: 9.5460e-04 - mae: 0.0220 - val_loss: 8.3437e-04 - val_mae: 0.0203
Epoch 5/5
731/731 ━━━━━━━━━━━━━━━━━━━━ 41s 35ms/step - loss: 7.4152e-04 - mae: 0.0201 - val_loss: 8.0653e-04 - val_mae: 0.0192


In [79]:
lstm_predictions = lstm_model.predict(X_test)
y_test_inverse = scaler.inverse_transform(y_test)
lstm_predictions_inverse = scaler.inverse_transform(lstm_predictions)
from sklearn.metrics import mean_squared_error
lstm_error = mean_squared_error(y_test_inverse, lstm_predictions_inverse)
print(f'LSTM MSE: {lstm_error}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
LSTM MSE: 215.1879226441248
